In [198]:
import pdb
import os
from pathlib import Path
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import math

In [197]:
# os.chdir('/home/tkf/Desktop/Uni/SWP/pollen-detection')
# os.chdir('..')
os.getcwd()

'/home/tkf/Desktop/Uni/SWP/pollen-detection'

In [199]:
"""
Prepare Data:
Create directories, annotation.json in COCO format and
crop raw images for pollen as well as foragers around real and fake annotationpoints
"""

DATA_CREATED = False
IMG_POLLEN_SIZE = 32
IMG_FORAGER_SIZE = 200
POLLEN_DIR = 'Data/PollenData/Real'
FORAGER_DIR = 'Data/ForagerData/Real'
RND_POLLEN_DIR = 'Data/PollenData/Fake'
RND_FORAGER_DIR = 'Data/ForagerData/Fake'
RAW_DIR = 'raw_data/BeesBook'
IMG_DIR = 'img'
ANN_DIR = 'ann'

if not os.path.isdir('Data'):
    os.makedirs(POLLEN_DIR)
    os.makedirs(FORAGER_DIR)
    os.mkdir(RND_POLLEN_DIR)
    os.mkdir(RND_FORAGER_DIR)


In [200]:
def eukl_dist(p1, p2):
    d = math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2))
    return d < IMG_FORAGER_SIZE

In [201]:
def create_cropped_img(filename, coords, img_counter, anno_counter, orig_dir):
    try:
        img = cv2.imread(filename, 0)
    except Exception as e:
        print(filename, str(e))
    
    for coord in coords:
        
        # random point
        too_close = True
        while too_close:
            rnd_x = random.randint(180, 2635)
            if rnd_x > 2360:
                coin = random.randint(0,1)
                if coin:
                    rnd_y = random.randint(90, 600)
                else:
                    rnd_y = random.randint(3500, 3950)
            else:
                rnd_y = random.randint(230, 2675)
            rnd_coord = [rnd_y, rnd_x]
            for coor in coords:
                too_close = eukl_dist(rnd_coord,coor)
                if too_close:
                    break
        
        # Pollen:
        time1 = str(datetime.datetime.utcnow())
        filename1 = (str(img_counter)).zfill(10) + '.png'
        anno_image1 = {'id':img_counter,'width':IMG_POLLEN_SIZE,'height':IMG_POLLEN_SIZE,'file_name':filename1,'license':None,'flickr_url':None,'coco_url':None,'date_captured':time1}
        anno_anno1 = {'id':anno_counter,'image_id':img_counter,'category_id':1,'segmentation':[],'area':IMG_POLLEN_SIZE**2,'bbox':[],'iscrowd':0}
        y_p = coord[0]-(IMG_POLLEN_SIZE//2)
        x_p = coord[1]-(IMG_POLLEN_SIZE//2)
        h_p = IMG_POLLEN_SIZE
        w_p = IMG_POLLEN_SIZE
        pollen_crop_img = img[x_p:x_p+h_p, y_p:y_p+w_p]
        
        # random non-pollen
        rnd_poll_filename = str(img_counter+3000000000) + '.png'
        r_y_p = rnd_coord[0]-(IMG_POLLEN_SIZE//2)
        r_x_p = rnd_coord[1]-(IMG_POLLEN_SIZE//2)
        rnd_pollen_crop_img = img[r_x_p:r_x_p+h_p, r_y_p:r_y_p+w_p]
        
        try:
            os.chdir('../../../../' + POLLEN_DIR)
            cv2.imwrite(filename1, pollen_crop_img)
            os.chdir('../Fake')
            cv2.imwrite(rnd_poll_filename, rnd_pollen_crop_img)
        except Exception as e:
            print(filename1, rnd_poll_filename, str(e))
        
        img_counter += 1
        anno_counter += 1
        
        # Forager:
        time2 = str(datetime.datetime.utcnow())
        filename2 = (str(img_counter)).zfill(10) + '.png'
        anno_image2 = {'id':img_counter,'width':IMG_FORAGER_SIZE,'height':IMG_FORAGER_SIZE,'file_name':filename2,'license':None,'flickr_url':None,'coco_url':None,'date_captured':time2}
        anno_anno2 = {'id':anno_counter,'image_id':img_counter,'category_id':2,'segmentation':[],'area':IMG_FORAGER_SIZE**2,'bbox':[],'iscrowd':0}
        y_f = coord[0]-(IMG_FORAGER_SIZE//2)
        x_f = coord[1]-(IMG_FORAGER_SIZE//2)
        h_f = IMG_FORAGER_SIZE
        w_f = IMG_FORAGER_SIZE
        forager_crop_img = img[x_f:x_f+h_f, y_f:y_f+w_f]
        
        # random non-forager
        rnd_forager_filename = str(img_counter+6000000000) + '.png'
        r_y_f = rnd_coord[0]-(IMG_FORAGER_SIZE//2)
        r_x_f = rnd_coord[1]-(IMG_FORAGER_SIZE//2)
        rnd_forager_crop_img = img[r_x_f:r_x_f+h_f, r_y_f:r_y_f+w_f]
        
        try:
            os.chdir('../../../' + FORAGER_DIR)
            cv2.imwrite(filename2, forager_crop_img)
            os.chdir('../Fake')
            cv2.imwrite(rnd_forager_filename, rnd_forager_crop_img)
        except Exception as e:
            print(filename2, rnd_forager_filename, str(e))
        
        img_counter += 1
        anno_counter += 1
        
        os.chdir(orig_dir)
        
    return (anno_image1, anno_anno1, anno_image2, anno_anno2, img_counter, anno_counter)

In [202]:
if not DATA_CREATED:
    today = str(datetime.datetime.utcnow())
    annotation = {
        'info': {'year': 2020,'version': None,'description': 'Pollenforager Detection','contributor': 'Mara Kortenkamp, Tim Feige','url': 'https://github.com/marakortenkamp/pollen-detection','date_created': today},
        'images': [],
        'annotations': [],
        'licenses': {'id': None,'name': None,'url': None,},
        'category': [{'supercategory': 'oject','id': 1,'name': 'Pollen'},{'supercategory': 'animal','id': 2,'name': 'Bee'}]
        }
    total_img_counter, total_anno_counter = 0, 9000000000
#     pdb.set_trace()
    os.chdir('%s' % RAW_DIR)
    folders = next(os.walk('.'))[1]
    for folder in folders:
        os.chdir(folder + '/' + ANN_DIR)
        files = os.listdir()
        for file in files:
            if 'json' in file:
                try:
                    with open(file) as json_file:
                        data = json.load(json_file)
                except Exception as e:
                    print(folder, file, str(e))
                if len(data['objects']):
                    try:
                        file_name = Path(file).stem
                        pollen_coords = []
                        for obj in data['objects']:
                            pollen_coords.append(obj['points']['exterior'][0])
                        os.chdir('../' + IMG_DIR)
                        current_dir = os.getcwd()
                    except Exception as e:
                        print(folder, file, obj, str(e))
                    try:
                        anno_img1, anno_anno1, anno_img2, anno_anno2, total_img_counter, total_anno_counter = create_cropped_img(file_name, pollen_coords, total_img_counter, total_anno_counter, current_dir)
                    except Exception as e:
                        print(folder, file, str(e))
                    try:
                        os.chdir('../' + ANN_DIR)
                    except Exception as e:
                        print(folder, file, str(e))
                    annotation['images'].append(anno_img1)
                    annotation['annotations'].append(anno_anno1)
                    annotation['images'].append(anno_img2)
                    annotation['annotations'].append(anno_anno2)
        os.chdir('../..')
    os.chdir('../..')    
    with open('annotation.json', 'w') as fp:
        json.dump(annotation, fp)

0000000019.png 6000000019.png OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

0000002129.png 6000002129.png OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

